https://influxdb-client.readthedocs.io/en/latest/usage.html#id4

or generate source code from Web UI： `Load Data` -> `Source`

## Query
Flux: https://docs.influxdata.com/influxdb/v2.0/query-data/flux/query-fields/

In [1]:
import re, os

path = '/data/conf/init.sh' # influxdb_token=... (optional)

if os.path.exists(path):
    with open(path,'r',encoding='utf-8') as f:
        content = f.read()
        token = re.search("(?<=influxdb_token=).*", content).group(0)
else:
    token = input("influxdb_token=")
    
from influxdb_client import InfluxDBClient

client = InfluxDBClient(url="http://192.168.88.21:8086", token=token, org="ferro")
query_api = client.query_api()    

In [2]:
import pandas as pd

query='''
from(bucket: "home")
  |> range(start: -3d)
  |> filter(fn: (r) => r["_measurement"] == "度数") 
  |> filter(fn: (r) => r["_field"] == "当前") 
  |> filter(fn: (r) => r["tag"] == "203" or r["tag"] == "205" )
  |> keep(columns: ["_time", "_value", "tag"])
'''

query = ''.join([ line for line in query.splitlines() if not line.startswith('#') ])

df  = client.query_api().query_data_frame(query=query).drop(['result', 'table'], axis=1)
df['_time'] =  pd.to_datetime(df['_time'], format='%Y-%m-%d %H:%M:%S.%f%z') # https://strftime.org/
df['tag'] = df['tag'].astype('string')

df['diff_value'] = df.groupby(['tag'])['_value'].diff().fillna(0)
df['diff_time']  = df.groupby(['tag'])['_time'].diff()

In [3]:
df.dtypes

_time         datetime64[ns, UTC]
_value                    float64
tag                        string
diff_value                float64
diff_time         timedelta64[ns]
dtype: object

In [4]:
df_last = df.groupby(['tag']).last()
df_last

,_time,_value,diff_value,diff_time
tag,,,,
203,2021-02-13 01:00:08.576459+00:00,1397.19,1.25,1 days 00:00:00.312928
205,2021-02-13 01:00:08.576459+00:00,185.72,1.77,1 days 00:00:00.312928


In [5]:
df_result = df[df.diff_time.notnull()]
df_result

,_time,_value,tag,diff_value,diff_time
1,2021-02-11 17:53:55.881142+00:00,1395.56,203,3.89,0 days 16:53:48.762028
2,2021-02-12 01:00:08.263531+00:00,1395.94,203,0.38,0 days 07:06:12.382389
3,2021-02-13 01:00:08.576459+00:00,1397.19,203,1.25,1 days 00:00:00.312928
5,2021-02-11 17:53:55.881142+00:00,183.62,205,2.51,0 days 16:53:48.762028
6,2021-02-12 01:00:08.263531+00:00,183.95,205,0.33,0 days 07:06:12.382389
7,2021-02-13 01:00:08.576459+00:00,185.72,205,1.77,1 days 00:00:00.312928
